In [1]:
!pip install -U transformers
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 87.7 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.2
    Uninstalling transformers-4.46.2:
      Successfully uninstalled transformers-4.46.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that a

In [2]:
!pip install huggingface-hub

In [3]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: write).
The token `token_write` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate wh

In [4]:
import torch
from transformers import (
    pipeline,
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
)
from datasets import load_dataset
import numpy as np
from typing import Dict, List, Tuple
import logging
import os

In [5]:
# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [6]:
# Model configurations
BERT_MODELS = [
    "lyeonii/bert-tiny",
    "lyeonii/bert-small",
    "lyeonii/bert-medium",
    "google-bert/bert-base-uncased",
    "google-bert/bert-large-uncased"
]

In [7]:
from transformers import TFAutoModelForSequenceClassification, FlaxAutoModelForSequenceClassification
from transformers import BertForSequenceClassification, RobertaForSequenceClassification
from huggingface_hub import Repository

class ToxicSpansAnalyzer:
    def __init__(self, model_name: str, dataset_name: str = 'heegyu/toxic-spans'):
        """
        Initialize the ToxicSpansAnalyzer with a specific model and dataset.

        Args:
            model_name: Name of the pretrained model to use
            dataset_name: Name of the dataset to use for fine-tuning
        """
        self.model_name = model_name
        self.dataset_name = dataset_name
        self.dataset = load_dataset(dataset_name)
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = None
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    def preprocess_dataset(self) -> Dict:
        """
        Preprocess the dataset for training.

        Returns:
            Preprocessed dataset dictionary
        """
        def preprocess_function(examples):
            return self.tokenizer(
                examples["text_of_post"],
                truncation=True,
                padding="max_length",
                max_length=128,
                return_attention_mask=True
            )

        def preprocess_labels(examples):
            examples["labels"] = examples["toxic"]
            return examples

        # Process the dataset
        tokenized_datasets = self.dataset.map(preprocess_function, batched=True)
        tokenized_datasets = tokenized_datasets.map(preprocess_labels)

        # Remove unnecessary columns
        columns_to_remove = [
            "text_of_post", "toxic", "probability", "position",
            "type", "support", "position_probability"
        ]
        tokenized_datasets = tokenized_datasets.remove_columns(columns_to_remove)

        return tokenized_datasets

    def fine_tune(self, output_dir: str):
        """
        Fine-tune the model on the toxic spans dataset and save the model files locally in PyTorch, TensorFlow, and Flax formats.
        Push all files to the Hugging Face Hub in one go.

        Args:
            output_dir: Directory to save the model and results
        """

        tokenized_datasets = self.preprocess_dataset()

        # Initialize model for sequence classification based on model name
        if "bert" in self.model_name.lower():
            self.model = BertForSequenceClassification.from_pretrained(
                self.model_name,
                num_labels=2,  # Binary classification
            ).to(self.device)
        elif "roberta" in self.model_name.lower():
            self.model = RobertaForSequenceClassification.from_pretrained(
                self.model_name,
                num_labels=2,  # Binary classification
            ).to(self.device)
        else:
            raise ValueError(f"Unsupported model type for {self.model_name}")

        # Define training arguments
        training_args = TrainingArguments(
            output_dir=output_dir,
            evaluation_strategy="epoch",
            save_strategy="epoch",
            learning_rate=2e-5,
            per_device_train_batch_size=16,
            per_device_eval_batch_size=64,
            num_train_epochs=3,
            weight_decay=0.01,
            logging_dir=f"{output_dir}/logs",
            logging_steps=10,
            save_total_limit=2,
            fp16=torch.cuda.is_available(),
        )

        # Initialize trainer
        trainer = Trainer(
            model=self.model,
            args=training_args,
            train_dataset=tokenized_datasets["train"],
            eval_dataset=tokenized_datasets["test"],
            tokenizer=self.tokenizer,
        )

        # Train the model
        trainer.train()

        self.model.save_pretrained(output_dir, safe_serialization=False)  # Save PyTorch model
        self.tokenizer.save_pretrained(output_dir)  # Save tokenizer files

        tf_model = TFAutoModelForSequenceClassification.from_pretrained(output_dir, from_pt=True)
        tf_model.save_pretrained(output_dir)  # Generates tf_model.h5

        flax_model = FlaxAutoModelForSequenceClassification.from_pretrained(output_dir, from_pt=True)
        flax_model.save_pretrained(output_dir)  # Generates flax_model.msgpack

        # Consolidate and push all files to the Hugging Face Hub
        print("Pushing all files to Hugging Face Hub in one go...")

        return trainer

In [8]:
def train_models(models: List[str], base_output_dir: str = "./results"):
    """
    Run the full experiment across multiple models.

    Args:
        models: List of model names to evaluate
        test_texts: List of texts to use for evaluation
        base_output_dir: Base directory for saving results
    """
    results = {}

    for model_name in models:
        logger.info(f"Processing model: {model_name}")

        try:
            # Initialize analyzer
            analyzer = ToxicSpansAnalyzer(model_name)

            # Fine-tune model
            output_dir = os.path.join(base_output_dir, model_name.replace('/', '_'))
            analyzer.fine_tune(output_dir)

        except Exception as e:
            logger.error(f"Error processing model {model_name}: {str(e)}")
            results[model_name] = {"error": str(e)}

    return results

In [9]:
bert_results = train_models(BERT_MODELS)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/3.65k [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/9.71M [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/954k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10006 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10006 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-large-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-7-7ddf6fadf60c>:96: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
ERROR:__main__:Error processing model google-bert/bert-large-uncased: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 19.06 MiB is free. Process 6391 has 14.73 GiB memory in use. Of the allocated memory 2.50 GiB is allocated by